In [1]:
import sys

sys.path.append('../../../')
import glob, os
import pandas as pd
from el_evaluation import *
import numpy as np


In [2]:
## data formatting

df_train = pd.read_csv("data/lcquad/lcquad_blink_train.csv")
print(df_train.shape)
df_train["Mention"] = df_train.Mention_label.str.split(';').str[0]
df_train['QuestionMention'] = df_train.Question + '--' + df_train.Mention
df_train = df_train.sort_values(by=['QuestionMention', 'Label'])
df_train.to_csv("data/lcquad/lcquad_train_sorted.csv")


df_test = pd.read_csv("data/lcquad/lcquad_blink_test.csv")
print(df_test.shape)
df_test["Mention"] = df_test.Mention_label.str.split(';').str[0]
df_test['QuestionMention'] = df_test.Question + '--' + df_test.Mention
df_test = df_test.sort_values(by=['QuestionMention', 'Label'])
df_test.to_csv("data/lcquad/lcquad_test_sorted.csv")

(318020, 5)
(32536, 5)


In [3]:
## data formatting

df_train = pd.read_csv("data/lcquad/lcquad_blink_train.csv")
print(df_train.shape)
df_test = pd.read_csv("data/lcquad/lcquad_blink_test.csv")
print(df_test.shape)
df_data = pd.concat([df_train, df_test])
df_data["Mention"] = df_data.Mention_label.str.split(';').str[0]
df_data['QuestionMention'] = df_data.Question + '--' + df_data.Mention
df_data.head()

(318020, 5)
(32536, 5)


,Unnamed: 0,Question,Mention_label,Features,Label,Mention,QuestionMention
0,0,What is the place where the mayor's wife is Je...,Jean-Marc Germain;Jean-Marc Germain (politician),"[0.9133332967758179, 0.6666666666666666, 0.566...",0,Jean-Marc Germain,What is the place where the mayor's wife is Je...
1,1,What is the place where the mayor's wife is Je...,Jean-Marc Germain;Jean-Marc Germain (businessman),"[0.9096774458885193, 0.6666666666666666, 0.548...",0,Jean-Marc Germain,What is the place where the mayor's wife is Je...
2,2,What is the place where the mayor's wife is Je...,Jean-Marc Germain;List of French supercentenar...,"[0.5002450942993164, 0.0, 0.25, 0.579229547897...",0,Jean-Marc Germain,What is the place where the mayor's wife is Je...
3,3,What is the place where the mayor's wife is Je...,Jean-Marc Germain;Jean-Claude Germain,"[0.9076986312866211, 0.3333333333333333, 0.736...",0,Jean-Marc Germain,What is the place where the mayor's wife is Je...
4,4,What is the place where the mayor's wife is Je...,Jean-Marc Germain;2005 French riots,"[0.5126050710678101, 0.0, 0.05882352941176472,...",0,Jean-Marc Germain,What is the place where the mayor's wife is Je...


In [4]:
df_data = df_data.sort_values(by=['QuestionMention', 'Label'])
df_data.to_csv("data/lcquad/lcquad_train_test_sorted.csv")
df_data.shape

(350556, 7)

In [5]:
from sklearn.model_selection import KFold
from pathlib import Path

df_data = pd.read_csv("data/lcquad/lcquad_train_test_sorted.csv")
question_list = np.array(df_data.Question.unique())
questions_idxes = list(range(len(df_data.Question.unique())))

kf = KFold(n_splits=5)
import os
average_performance = []
fold_num = 1
for train_index, test_index in kf.split(questions_idxes):
#     args.output_file_name = "output/complex_kfolds_{}.csv".format(fold_num)
    train_ques_set = question_list[train_index]
    test_ques_test = question_list[test_index]

    df_train_split = df_data[df_data.Question.isin(train_ques_set)]
    df_test_split = df_data[df_data.Question.isin(test_ques_test)]
    
    # filter out the questions with single positive or many negatives in trianing set
    filtered_question_mentions = []
    for qm in df_train_split.QuestionMention.unique():
        df_ = df_train_split[df_train_split.QuestionMention == qm]
        if df_.Label.sum() == 0:
            filtered_question_mentions.append(qm)
        if df_.Label.sum() == 1 and df_.shape[0] == 1:
            filtered_question_mentions.append(qm)
#             print(df_.Label.values)
    df_train_split_filtered = df_train_split[~df_train_split.QuestionMention.isin(filtered_question_mentions)]
    df_train_split_filtered = df_train_split_filtered.sort_values(by=['QuestionMention', 'Label'])
#     print(list(df_train_split_filtered.Label.values))
    
    print("=========== fold num --", fold_num, "===========")
    print("Before filtering train set:")
    print('Train', 'QuestionMention pairs -', df_train_split.shape[0], 'Question -', df_train_split.Question.unique().shape[0])
    print("After filtering train set:")
    print('Train', 'QuestionMention pairs -', df_train_split_filtered.shape[0], 'Question -', df_train_split_filtered.Question.unique().shape[0])
    
    print( 'Test', 'QuestionMention pairs -', df_test_split.shape[0], 'Question -', df_test_split.Question.unique().shape[0])

# #     os.mkdir('data/qald-5-folds/split{}'.format(fold_num))
    
#     df_missing = pd.read_csv("data/missing.csv", header=None)
#     df_missing.columns = ['Unnamed:0', 'Question', 'Entities', 'Classes']
#     df_missing = df_missing[['Question', 'Entities', 'Classes']]
    Path('data/lcquad/lcquad-5-folds/split{}'.format(fold_num)).mkdir(parents=True, exist_ok=True)
    df_test_split.to_csv('data/lcquad/lcquad-5-folds/split{}/test_fold.csv'.format(fold_num))
    df_train_split.to_csv('data/lcquad/lcquad-5-folds/split{}/train_fold.csv'.format(fold_num))
#     df_missing.to_csv('data/type-qald-5-folds/split{}/missing.csv'.format(fold_num))
#     print( 'Test + missing questions', 'question pairs -', df_test.shape[0]+df_missing.shape[0], 
#           'questions -', df_test.Question.unique().shape[0]+df_missing.Question.shape[0])
    fold_num += 1

=========== fold num -- 1 ===========
Before filtering train set:
Train QuestionMention pairs - 276361 Question - 3144
After filtering train set:
Train QuestionMention pairs - 243332 Question - 2858
Test QuestionMention pairs - 74195 Question - 786
=========== fold num -- 2 ===========
Before filtering train set:
Train QuestionMention pairs - 275536 Question - 3144
After filtering train set:
Train QuestionMention pairs - 242397 Question - 2848
Test QuestionMention pairs - 75020 Question - 786
=========== fold num -- 3 ===========
Before filtering train set:
Train QuestionMention pairs - 284364 Question - 3144
After filtering train set:
Train QuestionMention pairs - 249440 Question - 2857
Test QuestionMention pairs - 66192 Question - 786
=========== fold num -- 4 ===========
Before filtering train set:
Train QuestionMention pairs - 282068 Question - 3144
After filtering train set:
Train QuestionMention pairs - 250622 Question - 2866
Test QuestionMention pairs - 68488 Question - 786
====

In [74]:
import argparse
from collections import defaultdict
import sys

sys.path.append('../../../')
from el_evaluation import *
from utils import MyBatchSampler
from torch.utils.data import DataLoader, TensorDataset
from utils import QuestionSampler
import copy
import torch
import random
from sklearn.model_selection import KFold
from pathlib import Path
random.seed(103)

device = torch.device("cpu")

parser = argparse.ArgumentParser(description="main training script for training lnn entity linking models")
parser.add_argument("--checkpoint_name", type=str, default="checkpoint/best_model.pt", help="checkpoint path")
parser.add_argument("--log_file_name", type=str, default="log.txt", help="log_file_name")
parser.add_argument("--experiment_name", type=str, default="exp_test", help="checkpoint path")
parser.add_argument("--model_name", type=str, default="purename", help="which model we choose")
parser.add_argument('--alpha', type=float, default=0.9, help='alpha for LNN')
parser.add_argument('--num_epoch', type=int, default=100, help='training epochs for LNN')
parser.add_argument("--use_fixed_threshold", action="store_true", help="default is to use binary`, otherwise use stem")
parser.add_argument("--use_kfolds", action="store_true", help="default is to use binary`, otherwise use stem")
parser.add_argument('--margin', type=float, default=0.601, help='margin for MarginRankingLoss')
parser.add_argument('--learning_rate', type=float, default=0.001, help='learning rate')
parser.add_argument("-f")  # to avoid breaking jupyter notebook
args = parser.parse_args()


In [75]:
from models.RuleLNN_nway_sigmoid_vec_type import *

In [76]:
def get_qald_metrics(pred_, m_labels_, ques_, mode='val'):
    """pred_ are 0/1 s after applying a threshold"""
    rows = []
    question_rows_map = {}
    question_mention_set = set()
    for i, pred in enumerate(pred_):
        pred = pred.data.tolist()[0]
        question = ques_[i]
        if question not in question_rows_map:
            question_rows_map[ques_[i]] = []
        if pred:
            men_entity_label = '_'.join(m_labels_[i].split(';')[-1].split())
            men_entity_mention = '_'.join(m_labels_[i].split(';')[0].split())
            if '-'.join([question, men_entity_mention]) in question_mention_set:
                question_rows_map[ques_[i]][-1].add(
                    ('http://dbpedia.org/resource/{}'.format(men_entity_label), pred))
            else:
                question_mention_set.add('-'.join([question, men_entity_mention]))
                question_rows_map[ques_[i]].append(set())
                question_rows_map[ques_[i]][-1].add(
                    ('http://dbpedia.org/resource/{}'.format(men_entity_label), pred))
    for key, preds_list_mentions in question_rows_map.items():
        if len(preds_list_mentions) > 1:
            rows.append([key, []])
            for preds_set in preds_list_mentions:
                sorted_values = sorted(list(preds_set), key=lambda x: x[1], reverse=True)[:5]
                rows[-1][1].append(sorted_values)
        elif len(preds_list_mentions) == 1:
            sorted_values = sorted(list(preds_list_mentions[0]), key=lambda x: x[1], reverse=True)[:5]
            rows.append([key, [sorted_values]])
        else:
            rows.append([key, []])

    df_output = pd.DataFrame(rows, columns=['Question', 'Entities'])
    df_output['Classes'] = str([])

    # gold
    benchmark = pd.read_csv('../../../data/gt_sparql.csv')
    benchmark = benchmark.set_index('Question')
    benchmark = benchmark.replace(np.nan, '', regex=True)
    benchmark['Entities'] = benchmark['Entities'].astype(object)
    is_qald_gt = True

    # pred
    predictions = df_output
    # print(df_output.shape)
    predictions = predictions.set_index('Question')
    predictions['Entities'] = predictions['Entities']
    predictions['Classes'] = predictions['Classes']

    metrics = compute_metrics(benchmark=benchmark, predictions=predictions, limit=410, is_qald_gt=is_qald_gt,
                              eval='full')

    scores = metrics['macro']['named']
    prec, recall, f1 = scores['precision'], scores['recall'], scores['f1']
    return prec, recall, f1, df_output


In [13]:
def evaluate_ranking_loss(eval_model, x_, y_, m_labels_, ques_, loss_fn, mode='val'):
    """evaluate a model on validation data"""
    eval_model.eval()

    dataset_ = TensorDataset(x_, y_)
    question_sampler = QuestionSampler(torch.utils.data.SequentialSampler(range(len(y_))), y_, False)
    loader = DataLoader(dataset_, batch_sampler=question_sampler, shuffle=False)
    total_loss = 0.0

    with torch.no_grad():
        pred_ = eval_model(x_, m_labels_)

        batch_num = 0
        for xb, yb in loader:
            if yb.shape[0] == 1:
                # print(xb, yb)
                continue
            yhat = eval_model(xb, yb)
            yb = yb.reshape(-1, 1)
            yhat_pos = yhat[-1].repeat((len(yhat) - 1), 1)
            yhat_neg = yhat[:-1]
            loss = loss_fn(yhat_pos, yhat_neg, torch.ones((len(yhat) - 1), 1).to(device))
            total_loss += loss.item() * (yb.shape[0] - 1)
            batch_num += 1
        avg_loss = total_loss / batch_num
        prec, recall, f1, _ = get_qald_metrics(pred_, m_labels_, ques_, mode=mode)  # train and val both use 'val' mode

    return avg_loss, f1, pred_


In [14]:
def train(model, train_data, val_data, test_data, checkpoint_name, num_epochs, margin, learning_rate):
    """train model and tune on validation set"""

    # unwrapping the data
    x_train, y_train, m_labels_train, ques_train = train_data
    x_val, y_val, m_labels_val, ques_val = val_data
    x_test, y_test, m_labels_test, ques_test = test_data

    # initialize the loss function and optimizer
    loss_fn = nn.MarginRankingLoss(margin=margin)  # MSELoss(), did not work neither
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    best_val_f1, best_val_loss = 0, 100000
    best_train_val_loss = 1000000
    # batch_size = 32

    # stats before training
    print("=========BEFORE TRAINING============")
    train_loss, train_f1, train_pred = evaluate_ranking_loss(model, x_train, y_train, m_labels_train, ques_train,
                                                             loss_fn)
    print("Train -- loss is {}; F1 is {}".format(train_loss, train_f1))
    val_loss, val_f1, val_pred = evaluate_ranking_loss(model, x_val, y_val, m_labels_val, ques_val, loss_fn)
    print("Val --  loss is {}; F1 is {}".format(val_loss, val_f1))
    test_loss, test_f1, test_pred = evaluate_ranking_loss(model, x_test, y_test, m_labels_test, ques_test, loss_fn,
                                                          mode='test')
    print("Test -- loss is {}; F1 is {}".format(test_loss, test_f1))

    # start training
    print("=========TRAINING============")
    dataset_train = TensorDataset(x_train, y_train)
    question_sampler = QuestionSampler(torch.utils.data.SequentialSampler(range(len(y_train))), y_train, False)
    loader = DataLoader(dataset_train, batch_sampler=question_sampler, shuffle=False)
    # loader = DataLoader(dataset_train, sampler=torch.utils.data.SequentialSampler(dataset_train), batch_size=batch_size, shuffle=False)  # always False
    # loader = DataLoader(dataset_train, sampler=torch.utils.data.WeightedRandomSampler(torch.FloatTensor([1, 100]), len(x_train), replacement=True), batch_size=64, shuffle=False)  # always False
    best_model = None

    for epoch in range(num_epochs):
        total_loss = 0.0
        idx = 0

        # prev xb
        prev_xb, prev_yb = None, None
        for xb, yb in loader:
            idx += 1
            model.train()  # set train to true
            optimizer.zero_grad()
            # handle question with multiple positives
            if yb.shape[0] == 1:
                yb = prev_yb
                # print(prev_xb)
                # print(xb)
                prev_xb[-1, :] = xb
                xb = prev_xb
                # print(xb)
            yhat = model(xb, yb)
            yb = yb.reshape(-1, 1)
            yhat_pos = yhat[-1].repeat((len(yhat) - 1), 1)
            yhat_neg = yhat[:-1]  # torch.mean(yhat[:-1], dim=0)

            loss = loss_fn(yhat_pos, yhat_neg, torch.ones((len(yhat) - 1), 1).to(device))
            if idx == 212:
                print('loss', idx, loss)

            total_loss += loss.item() * (yb.shape[0] - 1)
            loss.backward()
            optimizer.step()
            prev_xb, prev_yb = xb, yb

        for name, param in model.named_parameters():
            if param.requires_grad:
                print(name, 'param -- data', param.data, 'grad -- ', param.grad)

        # show status after each epoch
        avg_loss = total_loss / idx
        #         train_loss, train_f1, train_pred = evaluate(model, x_train, y_train, m_labels_train, ques_train, loss_fn)
        print("Epoch " + str(epoch) + ": avg train loss -- " + str(avg_loss))
        #         print("Train -- loss is {}; F1 is {}".format(train_loss, train_f1))
        val_loss, val_f1, val_pred = evaluate_ranking_loss(model, x_val, y_val, m_labels_val, ques_val, loss_fn)

        if val_f1 >= best_val_f1:
            # if val_f1 >= best_val_f1:
            best_val_f1 = val_f1
            best_val_loss = val_loss
            # best_train_val_loss = val_loss + avg_loss
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), checkpoint_name)
        print("Val --  best loss is {}; F1 is {}".format(best_val_loss, best_val_f1))

        test_loss, test_f1, test_pred = evaluate_ranking_loss(model, x_test, y_test, m_labels_test, ques_test, loss_fn,
                                                              mode='test')
        print("Current Test -- loss is {}; F1 is {}".format(test_loss, test_f1))

    # show stats after training
    print("=========AFTER TRAINING============")
    train_loss, train_f1, train_pred = evaluate_ranking_loss(best_model, x_train, y_train, m_labels_train, ques_train,
                                                             loss_fn)
    print("Train -- loss is {}; F1 is {}".format(train_loss, train_f1))
    val_loss, val_f1, val_pred = evaluate_ranking_loss(best_model, x_val, y_val, m_labels_val, ques_val, loss_fn)
    print("Val --  loss is {}; F1 is {}".format(val_loss, val_f1))
    test_loss, test_f1, test_pred = evaluate_ranking_loss(best_model, x_test, y_test, m_labels_test, ques_test, loss_fn,
                                                          mode='test')
    print("Test -- loss is {}; F1 is {}".format(test_loss, test_f1))

    return


In [15]:
def test(x_test, m_labels_test, ques_test, alpha, checkpoint_name, model_name, output_file_name):
    """make predictions on test set"""
    bestModel = pick_model(model_name, alpha)
    bestModel.load_state_dict(torch.load(checkpoint_name))
    bestModel.eval()
    best_scores = {}

    with torch.no_grad():
        test_pred = bestModel(x_test, m_labels_test)
        prec, recall, f1, df_output = get_qald_metrics(test_pred, m_labels_test, ques_test, mode='test')
        df_output.to_csv(output_file_name)
        print("Test -- f1 is {} ".format(f1))
        print("Test -- prec, recall, f1", prec, recall, f1)
        best_scores['precision'] = prec
        best_scores['recall'] = recall
        best_scores['f1'] = f1

    # for name, mod in bestModel.named_modules():
    #     if type(mod) == nn.ModuleList:
    #         for name1, mod1 in mod.named_modules():
    #             if 'cdd' not in name1 and 'AND' not in name1:
    #                 if 'batch' in name1.lower():
    #                     continue
    #                 elif 'or_max' in name1.lower():
    #                     continue
    #                 elif 'and' in name1.lower():
    #                     print(name1, mod1.cdd())
    #                 elif 'or' in name1.lower():
    #                     print(name1, mod1.AND.cdd())
    #     else:
    #         if 'cdd' not in name and 'AND' not in name:
    #             if 'batch' in name.lower():
    #                 continue
    #             elif 'or_max' in name.lower():
    #                 continue
    #             elif 'and' in name.lower():
    #                 print(name, mod.cdd())
    #             elif 'or' in name.lower():
    #                 print(name, mod.AND.cdd())
    return test_pred, best_scores


In [16]:
def pick_model(model_name, alpha):
    if model_name == "purename":
        return PureNameLNN(alpha, -1, False)
    elif model_name == "context":
        return ContextLNN(alpha, -1, False)
    elif model_name == "complex":
        return ComplexRuleLNN(alpha, -1, False)
    else:
        print("WRONG name input")
        return None

In [ ]:
args.output_file_name = "output/complex_kfolds_{}.csv".format(fold_num)

df_train = pd.read_csv("./data/lcquad/type-lcquad-5-folds/split{}/train_fold.csv".format(fold_num))
df_test = pd.read_csv("./data/lcquad/type-lcquad-5-folds/split{}/test_fold.csv".format(fold_num))

# filter out the questions with single positive or many negatives in trianing set
filtered_question_mentions = []
for qm in df_train.QuestionMention.unique():
    df_ = df_train[df_train.QuestionMention == qm]
    if df_.Label.sum() == 0:
        filtered_question_mentions.append(qm)
    if df_.Label.sum() == 1 and df_.shape[0] == 1:
        filtered_question_mentions.append(qm)
#             print(df_.Label.values)
df_train_split_filtered = df_train[~df_train.QuestionMention.isin(filtered_question_mentions)]
df_train_split_filtered = df_train_split_filtered.sort_values(by=['QuestionMention', 'Label'])
df_train = df_train_split_filtered

# train
features_train = np.array(
    [np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_train.Features.values])
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(df_train.Label.values).float().reshape(-1, 1)
m_labels_train = df_train.Mention_label.values
ques_train = df_train.Question.values

# test
features_test = np.array(
    [np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_test.Features.values])
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(df_test.Label.values).float().reshape(-1, 1)
m_labels_test = df_test.Mention_label.values
ques_test = df_test.Question.values

# train model and evaluate
model = pick_model(args.model_name, args.alpha)

print(model)

print("model: ", args.model_name, args.alpha)
print(model(x_train, m_labels_train))

print("y_train sum", sum(y_train), sum(y_train) / len(y_train))
print("y_test sum", sum(y_test), sum(y_test) / len(y_test))

# aggregate the data into train, val, and test
train_data = (x_train, y_train, m_labels_train, ques_train)
print("train:", x_train.shape, y_train.shape, m_labels_train.shape, ques_train.shape)
test_data = (x_test, y_test, m_labels_test, ques_test)
print("test:", x_test.shape, y_test.shape, m_labels_test.shape, ques_test.shape)

# check class distribution
print("y_train sum", sum(y_train), sum(y_train) / len(y_train))
print("y_test sum", sum(y_test), sum(y_test) / len(y_test))

# train(model, train_data, test_data, test_data, args.checkpoint_name, args.num_epoch, args.margin,
#       args.learning_rate)
best_tuned_threshold = -1
test_pred, best_scores = test(x_test, m_labels_test, ques_test, best_tuned_threshold, args.alpha, args.checkpoint_name, args.model_name, args.output_file_name)
with open("output_w_spacy.txt", 'a') as f:
    f.write(
        "model={}; use_binary={}; alpha={}; p={}; r={}; f1={}\n".format(args.model_name,
                                                                                          args.use_binary,
                                                                                          args.alpha,
                                                                                          best_scores[
                                                                                              'precision'],
                                                                                          best_scores[
                                                                                              'recall'],
                                                                                          best_scores['f1']))
    print("model={}; use_binary={}; alpha={}; p={}; r={}; f1={}\n".format(args.model_name, args.use_binary,
                                                                          args.alpha,
                                                                          best_scores['precision'],
                                                                          best_scores['recall'],
                                                                          best_scores['f1']))
average_performance.append([best_scores['precision'], best_scores['recall'], best_scores['f1']])
fold_num += 1

In [20]:
# average_performance = []
# for fold_num in range(1, 6):
#     args.output_file_name = "output/complex_kfolds_{}.csv".format(fold_num)

#     df_train = pd.read_csv("./data/lcquad/type-lcquad-5-folds/split{}/train_fold.csv".format(fold_num))
#     df_test = pd.read_csv("./data/lcquad/type-lcquad-5-folds/split{}/test_fold.csv".format(fold_num))

#     # filter out the questions with single positive or many negatives in trianing set
#     filtered_question_mentions = []
#     for qm in df_train.QuestionMention.unique():
#         df_ = df_train[df_train.QuestionMention == qm]
#         if df_.Label.sum() == 0:
#             filtered_question_mentions.append(qm)
#         if df_.Label.sum() == 1 and df_.shape[0] == 1:
#             filtered_question_mentions.append(qm)
#     #             print(df_.Label.values)
#     df_train_split_filtered = df_train[~df_train.QuestionMention.isin(filtered_question_mentions)]
#     df_train_split_filtered = df_train_split_filtered.sort_values(by=['QuestionMention', 'Label'])
#     df_train = df_train_split_filtered
    
#     break
#     # train
#     features_train = np.array(
#         [np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_train.Features.values])
#     x_train = torch.from_numpy(features_train).float()
#     y_train = torch.from_numpy(df_train.Label.values).float().reshape(-1, 1)
#     m_labels_train = df_train.Mention_label.values
#     ques_train = df_train.Question.values

#     # test
#     features_test = np.array(
#         [np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_test.Features.values])
#     x_test = torch.from_numpy(features_test).float()
#     y_test = torch.from_numpy(df_test.Label.values).float().reshape(-1, 1)
#     m_labels_test = df_test.Mention_label.values
#     ques_test = df_test.Question.values

#     # train model and evaluate
#     model = pick_model(args.model_name, args.alpha)

#     print(model)

#     print("model: ", args.model_name, args.alpha)
#     print(model(x_train, m_labels_train))

#     print("y_train sum", sum(y_train), sum(y_train) / len(y_train))
#     print("y_test sum", sum(y_test), sum(y_test) / len(y_test))

#     # aggregate the data into train, val, and test
#     train_data = (x_train, y_train, m_labels_train, ques_train)
#     print("train:", x_train.shape, y_train.shape, m_labels_train.shape, ques_train.shape)
#     test_data = (x_test, y_test, m_labels_test, ques_test)
#     print("test:", x_test.shape, y_test.shape, m_labels_test.shape, ques_test.shape)

#     # check class distribution
#     print("y_train sum", sum(y_train), sum(y_train) / len(y_train))
#     print("y_test sum", sum(y_test), sum(y_test) / len(y_test))

#     # train(model, train_data, test_data, test_data, args.checkpoint_name, args.num_epoch, args.margin,
#     #       args.learning_rate)
#     best_tuned_threshold = -1
#     test_pred, best_scores = test(x_test, m_labels_test, ques_test, best_tuned_threshold, args.alpha, args.checkpoint_name, args.model_name, args.output_file_name)
#     with open("output_w_spacy.txt", 'a') as f:
#         f.write(
#             "model={}; use_binary={}; alpha={}; p={}; r={}; f1={}\n".format(args.model_name,
#                                                                                               args.use_binary,
#                                                                                               args.alpha,
#                                                                                               best_scores[
#                                                                                                   'precision'],
#                                                                                               best_scores[
#                                                                                                   'recall'],
#                                                                                               best_scores['f1']))
#         print("model={}; use_binary={}; alpha={}; p={}; r={}; f1={}\n".format(args.model_name, args.use_binary,
#                                                                               args.alpha,
#                                                                               best_scores['precision'],
#                                                                               best_scores['recall'],
#                                                                               best_scores['f1']))
#     average_performance.append([best_scores['precision'], best_scores['recall'], best_scores['f1']])
#     fold_num += 1

# average_performance = np.array(average_performance)
# print("Avg performance is prec - rec - f1: ", average_performance.mean(0))


Avg performance is prec - rec - f1:  nan


/Users/hjian42/opt/anaconda3/envs/entity/lib/python3.6/site-packages/ipykernel_launcher.py:82: RuntimeWarning: Mean of empty slice.
/Users/hjian42/opt/anaconda3/envs/entity/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [57]:
features_train = np.array(
    [np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_train.Features.values])


/Users/hjian42/opt/anaconda3/envs/entity/lib/python3.6/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [69]:
df_test.Features.str.split(",").apply(len)

0        10
1        10
2        10
3        10
4        10
         ..
16181    10
16182    10
16183    10
16184    10
16185    10
Name: Features, Length: 16186, dtype: int64

In [51]:
s = df_train.Features[0]
s

'[0.9066666960716248, 0.5, 0.5333333333333333, 0.7606099446321317, 1.0, 1.0, 8.0, 1.0, 1.0, 0]'

In [27]:
x_train = torch.from_numpy(features_train).float()

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
python run_el_kfolds_type.py \
--experiment_name exp_complex_pure_ctx_type \
--model_name complex_pure_ctx_type \
--num_epoch 25 \
--margin 0.601 \
--alpha 0.9 \
--learning_rate 0.001 \
--log_file_name exp_complex_pure_ctx_type.log \
--checkpoint_name checkpoint/best_model_complex_pure_ctx_type.pt